In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer # sklearn 안에 있는 패키지
from sklearn.metrics.pairwise import cosine_similarity # TF-IDF 값을 기준으로 코사인 유사도를 통해 가장 유사한 제목을 찾아주기떄문에 cosine_similarity를 사용

In [2]:
# 데이터를 데이터프레임으로 로드
data = pd.read_csv("../library_book_data.csv", encoding='utf-8')# 데이터 파일 경로

/tmp/ipykernel_21392/600411025.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../library_book_data.csv", encoding='utf-8')# 데이터 파일 경로


In [3]:
# Null 값은 빈 문자열로 대체
data.fillna("", inplace=True)
print(data)

/tmp/ipykernel_21392/3805302243.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.fillna("", inplace=True)


         SEQ_NO  ISBN_THIRTEEN_NO VLM_NM  \
0       6352228     9791156759270          
1       6352229     9791168120877          
2       6352230     9791168120839          
3       6352231     9791168120846          
4       6352232     9791168120747          
...         ...               ...    ...   
157113  6372497     9781191496148          
157114  6372498     9781161755015          
157115  6372499     9781197192907          
157116  6372500     9781188005384          
157117  6372501     9781191334426          

                                                 TITLE_NM  \
0            너에게 목소리를 보낼게 - &lt;달빛천사&gt; 성우 이용신의 첫 번째 에세이   
1                   일기에도 거짓말을 쓰는 사람 - 99년생 시인의 자의식 과잉 에세이   
2                             본격 한중일 세계사 12 - 임오군란과 통킹 위기   
3                    즉시 기분을 바꿔드립니다 - 신기하게 마음이 편해지는 응급 처방전   
4                               오늘도 리추얼 : 음악, 나에게 선물하는 시간   
...                                                   ...   
157113            설민석의 세계사 대모험 11 (영국 산업 혁명 

In [5]:
# TF-IDF 벡터화기 생성
tfidf_vectorizer = TfidfVectorizer()

In [11]:
# TF-IDF 매트릭스 계산
tfidf_matrix = tfidf_vectorizer.fit_transform(data["BOOK_INTRCN_CN"])

In [12]:
print(tfidf_matrix) 

  (0, 243763)	0.0538320077294149
  (0, 331904)	0.10779881667428079
  (0, 133501)	0.14340098386723107
  (0, 256580)	0.1750329632918931
  (0, 324920)	0.12651710141098219
  (0, 93465)	0.15908897391220073
  (0, 198100)	0.10148926091713605
  (0, 180866)	0.16785069717543477
  (0, 268913)	0.19289158915275378
  (0, 247883)	0.1294079563956494
  (0, 177362)	0.10249925165208779
  (0, 205605)	0.11233153883679733
  (0, 137665)	0.09365437924698565
  (0, 235547)	0.39896535320019355
  (0, 169806)	0.3538951995461228
  (0, 166501)	0.1882151381865791
  (0, 289644)	0.14013895693701903
  (0, 39376)	0.08620732842829222
  (0, 141923)	0.14560852235911173
  (0, 11509)	0.19948267660009678
  (0, 114857)	0.3538951995461228
  (0, 208367)	0.19289158915275378
  (0, 311970)	0.19289158915275378
  (0, 108504)	0.18162405073923607
  (0, 269162)	0.10811956580640111
  :	:
  (157111, 161693)	0.2208374952678868
  (157111, 333669)	0.2208374952678868
  (157111, 142536)	0.2208374952678868
  (157111, 128399)	0.2208374952678868
 

In [13]:
# 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

MemoryError: Unable to allocate 10.7 GiB for an array with shape (1438150989,) and data type float64

In [9]:
print(cosine_sim)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [10]:
# 추천할 도서 선택 (예시: 첫 번째 도서)
book_index = 2

In [ ]:
# 해당 도서와 유사한 도서 추천 (상위 5개)
similar_books = list(enumerate(cosine_sim[book_index]))
similar_books.sort(key=lambda x: x[1], reverse=True)
recommended_books = similar_books[1:6]